# Roe Approximate Riemann Solver Shallow Water Equations

$$
\renewcommand{\v}[1]{\mathbf{#1}}
% redefine integral commands with inputs
% indefinite integral without dx
\let\oldint\int
\newcommand{\inta}[1]{\oldint\!\paren{#1}}
\newcommand{\intna}[1]{\oldint\!{#1}}
\def\int{\@ifstar{\inta}{\intna}}
% indefinite integral with dx
\newcommand{\intta}[2]{\oldint\!\paren{#1}\,\mathrm{d}{#2}}
\newcommand{\inttna}[2]{\oldint\!{#1}\,\mathrm{d}{#2}}
\def\intt{\@ifstar{\intta}{\inttna}}
% definite integral with bounds but no dx
\newcommand{\dintna}[3]{\oldint_{#1}^{#2}\!{#3}}
\newcommand{\dinta}[3]{\oldint_{#1}^{#2}\!\paren{#3}}
\def\dint{\@ifstar{\dinta}{\dintna}}
% definite integral
\newcommand{\dinttna}[4]{\oldint_{#1}^{#2}\!{#3}\,\mathrm{d}{#4}}
\newcommand{\dintta}[4]{\oldint_{#1}^{#2}\!\paren{#3}\,\mathrm{d}{#4}}
\def\dintt{\@ifstar{\dintta}{\dinttna}}
\let\oldiint\iint
\newcommand{\iinta}[1]{\oldiint\!\paren{#1}}
\newcommand{\iintna}[1]{\oldiint\!{#1}}
\def\iint{\@ifstar{\iinta}{\iintna}}
% indefinite integral with dx
\newcommand{\iintta}[2]{\oldiint\!\paren{#1}\,\mathrm{d}{#2}}
\newcommand{\iinttna}[2]{\oldiint\!{#1}\,\mathrm{d}{#2}}
\def\iintt{\@ifstar{\iintta}{\iinttna}}
% definite integral with bounds but no dx
\newcommand{\diintna}[3]{\oldiint_{#1}^{#2}\!{#3}}
\newcommand{\diinta}[3]{\oldiint_{#1}^{#2}\!\paren{#3}}
\def\diint{\@ifstar{\diinta}{\diintna}}
% definite integral
\newcommand{\diinttna}[4]{\oldiint_{#1}^{#2}\!{#3}\,\mathrm{d}{#4}}
\newcommand{\diintta}[4]{\oldiint_{#1}^{#2}\!\paren{#3}\,\mathrm{d}{#4}}
\def\diintt{\@ifstar{\diintta}{\diinttna}}
% define macro for bounds
\newcommand{\evalb}[3]{\left. #1 \right\rvert_{#2}^{#3}}
\newcommand{\evall}[2]{\left. #1 \right\rvert_{#2}}
\def\eval{\@ifstar{\evalb}{\evall}}
$$

Conservation Law
$$ \v{q}_t + \v{f}(\v{q})_x = 0 $$


In [40]:
g = var('g')
# primitive variables
h = var('h')
u = var('u')
primitive = vector([h, u])
p = primitive

# f_p(p)
flux_primitive = vector([h*u, h*u^2 + 1/2 * g * h^2])
f_p = flux_primitive

# conserved variables
q_1 = var('q_1')
q_2 = var('q_2')
# q = [q_1, q_2]
conserved = vector([q_1, q_2])
q = conserved

# transform variables
z_1 = var('z_1')
z_2 = var('z_2')
transform = vector([z_1, z_2])
z = transform

# substitution dictionaries
primitive_to_conserved = {h:q_1, u: q_2/q_1}
p_to_q = primitive_to_conserved
primitive_to_transform = {h:z_1^2, u:z_2/z_1}
p_to_z = primitive_to_transform
conserved_to_primitive = {q_1:h, q_2: h*u}
q_to_p = conserved_to_primitive
conserved_to_transform = {q_1:z_1^2, q_2:z_1*z_2}
q_to_z = conserved_to_transform
transform_to_conserved = {z_1:sqrt(q_1), z_2:q_2/sqrt(q_1)}
z_to_q = transform_to_conserved
transform_to_primitive = {z_1:sqrt(h), z_2:sqrt(h)*u}
z_to_p = transform_to_primitive

# transformation symbolic expressions
# p_q(q)
primitive_from_conserved = vector([q_1, q_2/q_1])
p_q = primitive_from_conserved
# p_z(z)
primitive_from_transform = vector([z_1^2, z_2/z_1])
p_z = primitive_from_transform

# q_p(p)
conserved_from_primitive = vector([h, h*u])
q_p = conserved_from_primitive
# q_z(z)
conserved_from_transform = vector([z_1^2, z_1*z_2])
q_z = conserved_from_transform

# z_p(p)
transform_from_primitive = vector([sqrt(h), sqrt(h)*u])
z_p = transform_from_primitive
# z_q(q)
transform_from_conserved = vector([sqrt(q_1), q_2/sqrt(q_1)])
z_q = transform_from_conserved

# transformation jacobians
p_q_j = jacobian(p_q, q)
p_z_j = jacobian(p_z, z)
q_p_j = jacobian(q_p, p)
q_z_j = jacobian(q_z, z)
z_p_j = jacobian(z_p, p)
z_q_j = jacobian(z_q, q)

# conserved and transform fluxes
flux_conserved = flux_primitive.subs(p_to_q)
f_q = flux_conserved
flux_transform = flux_primitive.subs(p_to_z)
f_z = flux_transform

# flux_jacobians
flux_primitive_jacobian = jacobian(f_p, p)
f_p_j = flux_primitive_jacobian
flux_conserved_jacobian = jacobian(f_q, q)
f_q_j = flux_conserved_jacobian
flux_transform_jacobian = jacobian(f_z, z)
f_z_j = flux_transform_jacobian

# Riemann Problem
# P_i = [H_i, U_i]
H_i = var('H_i')
U_i = var('U_i')
p_to_P_i = {h: H_i, u: U_i}
P_i = p.subs(p_to_P_i)
# P_im1 = [H_im1, U_im1]
H_im1 = var('H_im1')
U_im1 = var('U_im1')
p_to_P_im1 = {h: H_im1, u: U_im1}
P_im1 = p.subs(p_to_P_im1)
# Q_i = [Q_1_i, Q_2_i]
Q_1_i = var('Q_1_i')
Q_2_i = var('Q_2_i')
q_to_Q_i = {q_1: Q_1_i, q_2: Q_2_i}
Q_i = q.subs(q_to_Q_i)
# Q_im1 = [Q_1_im1, Q_2_im1]
Q_1_im1 = var('Q_1_im1')
Q_2_im1 = var('Q_2_im1')
q_to_Q_im1 = {q_1: Q_1_im1, q_2: Q_2_im1}
Q_im1 = q.subs(q_to_Q_im1)
# Z_i = [Z_1_i, Z_2_i]
Z_1_i = var('Z_1_i')
Z_2_i = var('Z_2_i')
z_to_Z_i = {z_1: Z_1_i, z_2: Z_2_i}
Z_i = z.subs(z_to_Z_i)
# Z_im1 = [Z_1_im1, Z_2_im1]
Z_1_im1 = var('Z_1_im1')
Z_2_im1 = var('Z_2_im1')
z_to_Z_im1 = {z_1: Z_1_im1, z_2: Z_2_im1}
Z_im1 = z.subs(z_to_Z_im1)

P_i_to_Q_i = {H_i: Q_1_i, U_i: Q_2_i/Q_1_i}
P_i_to_Z_i = {H_i: Z_1_i^2, U_i: Z_2_i/Z_1_i}
Q_i_to_P_i = {Q_1_i: H_i, Q_2_i: H_i*U_i}
Q_i_to_Z_i = {Q_1_i: Z_1_i^2, Q_2_i:Z_1_i*Z_2_i}
Z_i_to_Q_i = {Z_1_i:sqrt(Q_1_i), Z_2_i: Q_2_i/sqrt(Q_1_i)}
Z_i_to_P_i = {Z_1_i:sqrt(H_i), Z_2_i: sqrt(H_i)*U_i}

P_im1_to_Q_im1 = {H_im1: Q_1_im1, U_im1: Q_2_im1/Q_1_im1}
P_im1_to_Z_im1 = {H_im1: Z_1_im1^2, U_im1: Z_2_im1/Z_1_im1}
Q_im1_to_P_im1 = {Q_1_im1: H_im1, Q_2_im1: H_im1*U_im1}
Q_im1_to_Z_im1 = {Q_1_im1: Z_1_im1^2, Q_2_im1: Z_1_im1*Z_2_im1}
Z_im1_to_Q_im1 = {Z_1_im1:sqrt(Q_1_im1), Z_2_im1: Q_2_im1/sqrt(Q_1_im1)}
Z_im1_to_P_im1 = {Z_1_im1:sqrt(H_im1), Z_2_im1: sqrt(H_im1)*U_im1}

# path substitution
xi = var('xi')
path_subs = {z_1: Z_1_im1 + (Z_1_i - Z_1_im1) * xi, z_2: Z_2_im1 + (Z_2_i - Z_2_im1) * xi}

# integrate over path
C = f_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
B = q_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
A = C*B.inverse()

assert(A[0, 0] == 0)
assert(A[0, 1] == 1)
# A[1, 1] = 2 u_bar
u_bar = (1/2*A[1, 1].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1)).simplify_full()
# A[1, 0] = gh - u^2
h_bar = ((A[1, 0] + u_bar^2)/g).subs(Z_i_to_P_i).subs(Z_im1_to_P_im1).simplify_full()

# check property A*(Q_i - Q_im1) == f_q(Q_i) - f_q(Q_im1)
test = (A.subs(Z_i_to_Q_i).subs(Z_im1_to_Q_im1)*(Q_i - Q_im1) - (f_q.subs(q_to_Q_i) - f_q.subs(q_to_Q_im1))).simplify_full()
assert(test[0] == 0)
assert(test[1] == 0)

In [30]:
u_bar

(sqrt(H_i)*U_i + sqrt(H_im1)*U_im1)/(sqrt(H_i) + sqrt(H_im1))

In [31]:
h_bar

1/2*H_i + 1/2*H_im1

In [73]:
g = var('g')
# primitive variables
h = var('h')
u = var('u')
s = var('s')
primitive = vector([h, u, s])
p = primitive

# f_p(p)
flux_primitive = vector([h*u, h*u^2 + 1/2 * g * h^2 + 1/3 * h * s^2, 2*h*u*s])
f_p = flux_primitive

# conserved variables
q_1 = var('q_1')
q_2 = var('q_2')
q_3 = var('q_3')
# q = [q_1, q_2]
conserved = vector([q_1, q_2, q_3])
q = conserved

# transform variables
z_1 = var('z_1')
z_2 = var('z_2')
z_3 = var('z_3')
transform = vector([z_1, z_2, z_3])
z = transform

# substitution dictionaries
primitive_to_conserved = {h:q_1, u: q_2/q_1, s: q_3/q_1}
p_to_q = primitive_to_conserved
primitive_to_transform = {h:z_1^2, u:z_2/z_1, s:z_3/z_1}
p_to_z = primitive_to_transform
conserved_to_primitive = {q_1:h, q_2: h*u, q_3: h*s}
q_to_p = conserved_to_primitive
conserved_to_transform = {q_1:z_1^2, q_2:z_1*z_2, q_3:z_1*z_3}
q_to_z = conserved_to_transform
transform_to_conserved = {z_1:sqrt(q_1), z_2:q_2/sqrt(q_1), z_3:q_3/sqrt(q_1)}
z_to_q = transform_to_conserved
transform_to_primitive = {z_1:sqrt(h), z_2:sqrt(h)*u, z_3:sqrt(h)*s}
z_to_p = transform_to_primitive


# transformation symbolic expressions
# p(q)
primitive_from_conserved = vector([q_1, q_2/q_1, q_3/q_1])
p_q = primitive_from_conserved
# p(z)
primitive_from_transform = vector([z_1^2, z_2/z_1, z_3/z_1])
p_z = primitive_from_transform

# q(p)
conserved_from_primitive = vector([h, h*u, h*s])
q_p = conserved_from_primitive
# q(z)
conserved_from_transform = vector([z1^2, z1*z2, z1*z3])
q_z = conserved_from_transform

# z(p)
transform_from_primitive = vector([sqrt(h), sqrt(h)*u, sqrt(h)*s])
z_p = transform_from_primitive
# z(q)
transform_from_conserved = vector([sqrt(q1), q2/sqrt(q1), q3/sqrt(q1)])
z_q = transform_from_conserved

# transformation jacobians
p_q_j = jacobian(p_q, q)
p_z_j = jacobian(p_z, z)
q_p_j = jacobian(q_p, p)
q_z_j = jacobian(q_z, z)
z_p_j = jacobian(z_p, p)
z_q_j = jacobian(z_q, q)

flux_conserved = flux_primitive.subs(p_to_q)
f_q = flux_conserved
flux_transform = flux_primitive.subs(p_to_z)
f_z = flux_transform

# flux_jacobians
flux_primitive_jacobian = jacobian(f_p, p)
f_p_j = flux_primitive_jacobian
flux_conserved_jacobian = jacobian(f_q, q)
f_q_j = flux_conserved_jacobian
flux_transform_jacobian = jacobian(f_z, z)
f_z_j = flux_transform_jacobian

# path substitution
xi = var('xi')
Z1im1 = var('Z_1_im1')
Z1i = var('Z_1_i')
Z2im1 = var('Z_2_im1')
Z2i = var('Z_2_i')
Z3im1 = var('Z_3_im1')
Z3i = var('Z_3_i')
path_subs = {z1: Z1im1 + (Z1i - Z1im1) * xi, z2: Z2im1 + (Z2i - Z2im1) * xi, z3: Z3im1 + (Z3i - Z3im1) * xi}

# integrate over path
C = f_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
B = q_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
A = C*B.inverse()

In [75]:
C.factor()

[                      1/2*Z_2_i + 1/2*Z_2_im1                       1/2*Z_1_i + 1/2*Z_1_im1                                             0]
[1/2*(Z_1_i^2 + Z_1_im1^2)*(Z_1_i + Z_1_im1)*g                               Z_2_i + Z_2_im1                       1/3*Z_3_i + 1/3*Z_3_im1]
[                                            0                               Z_3_i + Z_3_im1                               Z_2_i + Z_2_im1]

In [77]:
B.factor()

[        Z_1_i + Z_1_im1                       0                       0]
[1/2*Z_2_i + 1/2*Z_2_im1 1/2*Z_1_i + 1/2*Z_1_im1                       0]
[1/2*Z_3_i + 1/2*Z_3_im1                       0 1/2*Z_1_i + 1/2*Z_1_im1]

In [81]:
# Z_1_bar = 1/2(Z_1_i + Z_1_im1)
Z1b = var('Z_1_b')
# Z_2_bar = 1/2(Z_2_i + Z_2_im1)
Z2b = var('Z_2_b')
# Z_3_bar = 1/2(Z_3_i + Z_3_im1)
Z3b = var('Z_3_b')
# Z_1_i^2 = h_i
# hb = 1/2*(Z_1_i^2 + Z_1_im1^2)
hb = var('h_b')
C = matrix([[Z2b, Z1b, 0],[2 * hb * Z1b * g, 2 * Z2b, 2/3*Z3b],[0, 2*Z3b, 2*Z2b]])
B = matrix([[2*Z1b, 0, 0],[Z2b, Z1b, 0],[Z3b, 0, Z1b]])
A = C*B.inverse()
A

[                                            0                                             1                                             0]
[g*h_b - Z_2_b^2/Z_1_b^2 - 1/3*Z_3_b^2/Z_1_b^2                                 2*Z_2_b/Z_1_b                               2/3*Z_3_b/Z_1_b]
[                       -2*Z_2_b*Z_3_b/Z_1_b^2                                 2*Z_3_b/Z_1_b                                 2*Z_2_b/Z_1_b]

In [83]:
# Z_2_b/Z_1_b = ((Z_2_im1 + Z_2_i)/(Z_1_i + Z_1_im1)) = (sqrt(h_im1)u_im1 + sqrt(h_i)*ui)/(sqrt(h_i) + sqrt(h_im1)) = u_bar
# Z_3_b/Z_1_b = ((Z_3_im1 + Z_3_i)/(Z_1_i + Z_1_im1)) = (sqrt(h_im1)s_im1 + sqrt(h_i)*si)/(sqrt(h_i) + sqrt(h_im1)) = s_bar
sb = var('s_b')
A = matrix([[0, 1, 0],[g*hb - ub^2 - 1/3*sb^2, 2*ub, 2/3*sb],[-2*ub*sb, 2*sb, 2*ub]])

In [ ]:
[                                            0                                             1                                             0]
[g*h_b - Z_2_b^2/Z_1_b^2 - 1/3*Z_3_b^2/Z_1_b^2                                 2*Z_2_b/Z_1_b                               2/3*Z_3_b/Z_1_b]
[                       -2*Z_2_b*Z_3_b/Z_1_b^2                                 2*Z_3_b/Z_1_b                                 2*Z_2_b/Z_1_b]

In [80]:
f_q_j.subs(q_to_p)

[                  0                   1                   0]
[g*h - 1/3*s^2 - u^2                 2*u               2/3*s]
[             -2*s*u                 2*s                 2*u]